# ETL

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv("C:/Users/pablo/Desktop/proyecto_final/2017PurchasePricesDec.csv")

df.isnull().sum()


df.dropna(how='any', inplace=True)

df['Size'] = df['Size'].str.lower()

df['Size'].unique()


array(['750ml', '1000ml', '1750ml', '50ml', '375ml', '100ml 4 pk',
       '50ml 5 pk', '100ml', '200ml', '50ml 4 pk', '50ml 3 pk', '300ml',
       '200ml 4 pk', 'unknown', '750ml 2 pk', '250ml 4 pk', '1500ml',
       '3000ml', '5000ml', '4000ml', '187ml 4 pk', '150ml', '187ml',
       '500ml', '720ml', '650ml', '200ml 3 pk', '187ml 3 pk', '330ml',
       '250ml', '750ml + 3/', '18000ml', '180ml', '750ml + 4/', '6000ml',
       '1000ml 2 pk', '750ml 12 p', '750ml 6 pk', '20ml 5 pk',
       '375ml 2 pk', '20000ml', '50ml 12 pk', '750ml 3 pk', '375ml 3 pk',
       '750ml + 1/', '200ml 5 pk', '162.5ml', '400ml', '1100ml', '600ml',
       '19500ml', '560ml', '3750ml', '750ml  3', '9000ml'], dtype=object)

In [83]:
df_work = df.copy()

# Normalización
s = (df_work['Size'].astype('string').str.lower().str.strip()
       .str.replace(r'\s+', ' ', regex=True)
       .str.replace(r'\bpk\.?\b', ' pk', regex=True)     # "pk." -> " pk"
       .str.replace(r'(?<=\d)\s+p\b', ' pk', regex=True) # "12 p" -> "12 pk"
)


In [84]:
pat = (
    r'^\s*'
    r'(?P<ml_val>\d+(?:\.\d+)?)\s*ml'            # 50ml | 162.5ml | 1750ml
    r'(?:\s*(?P<qty>\d{1,3})(?:\s*pk)?)?'        # "12 pk" o "ml 12" 
    r'(?:\s*\+\s*(?P<free>\d+)\s*/)?'            # "+ 3/" 
    r'\s*$'
)
m = s.str.extract(pat)


In [85]:
# Construir las DOS columnas
#  'size_ml' como string "Nml" (redondeo por si aparece 162.5ml)
ml_num = (pd.to_numeric(m['ml_val'], errors='coerce').round().astype('Int64'))
size_ml = (ml_num.astype('string') + 'ml').where(ml_num.notna(), pd.NA)

In [86]:
qty_pk = pd.to_numeric(m['qty'],  errors='coerce').astype('Int64')
free   = pd.to_numeric(m['free'], errors='coerce').astype('Int64')


In [87]:
size_qty = qty_pk.where(qty_pk.notna(), free)      # si no hay pk, usa free
size_qty = size_qty.where(size_qty.notna(), 1)     # si no hay nada, 1
size_qty = size_qty.where(ml_num.notna(), pd.NA)   # si no hay tamaño, NA

In [111]:
# 4) Asignar al DataFrame final (solo dos columnas nuevas)
df_work['size_ml']  = size_ml
df_work['size_qty'] = size_qty




#### elimino columnas redundantes

In [89]:
df_work.drop(columns=['Size', 'size_ml'], errors='ignore', inplace=True)

#### relleno con 0 los na en volume

In [ ]:
#relleno con 0 por que son solo 4 filas, no causa problemas en mi analisis

df_work['Volume'] = df_work['Volume'].fillna(0)

In [ ]:
# Asegurar que sean numéricas (por si llegaron como texto)
df_work['Price']   = pd.to_numeric(df_work['Price'], errors='coerce')
df_work['PurchasePrice'] = pd.to_numeric(df_work['PurchasePrice'], errors='coerce')
df_work['Volume']  = pd.to_numeric(df_work['Volume'], errors='coerce').round().astype('Int64')


#creo la coluna 'gross_margin' (diferencia entre valor de compra y valor de venta)
df_work['gross_margin']  = df_work['Price'] - df_work['PurchasePrice']






0         750
1         750
2         750
3         750
4         750
         ... 
12256     750
12257     750
12258     750
12259    1500
12260     750
Name: Volume, Length: 12260, dtype: Int64

In [110]:
df_work.to_csv("C:/Users/pablo/Desktop/proyecto_final/2017PurchasePricesDec_limpio.csv")